In [1]:
from google.colab import files
uploaded = files.upload()

Exception in thread Thread-3 (attachment_entry):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.11/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise_exception("{0}() failed:", func.__name__, level="info")
  File "/usr/local/lib/python3.11/dist-packages/debugpy/server/api.py", line 141, in debug
    return func(address, settrace_kwargs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/debugpy/server/ap

Saving aqi.csv to aqi.csv
Saving vahan.csv to vahan.csv


In [3]:
import pandas as pd

# Load datasets
vahan_df = pd.read_csv("vahan.csv")
aqi_df = pd.read_csv("aqi.csv")

# Convert AQI date column
aqi_df['date'] = pd.to_datetime(aqi_df['date'], format="%d-%m-%Y", errors='coerce')

# Step 1: Filter for EVs in vehicle dataset (assuming 'fuel_type' column exists)
ev_df = vahan_df[vahan_df['fuel'].str.lower().str.contains('electric')]

# Step 2: Sum EV registrations per state
ev_counts = ev_df.groupby('state')['value'].sum().reset_index()
ev_counts.rename(columns={'value': 'ev_count'}, inplace=True)

# Step 3: Get Top 5 EV adoption states
top5_ev_states = ev_counts.sort_values(by='ev_count', ascending=False).head(5)
top5_states_list = top5_ev_states['state'].tolist()

# Step 4: Calculate average AQI per state
aqi_filtered = aqi_df[aqi_df['date'] >= '2022-01-01']
avg_aqi = aqi_filtered.groupby('state')['aqi_value'].mean().reset_index()
avg_aqi.rename(columns={'aqi_value': 'avg_aqi'}, inplace=True)

# Step 5: Merge EV and AQI data
ev_aqi_merged = pd.merge(ev_counts, avg_aqi, on='state', how='left')

# Add category: 'Top 5 EV States' or 'Others'
ev_aqi_merged['category'] = ev_aqi_merged['state'].apply(
    lambda x: 'Top 5 EV States' if x in top5_states_list else 'Others'
)

# Export to CSV
ev_aqi_merged.to_csv("EV_Adoption_vs_AQI.csv", index=False)

# Preview
ev_aqi_merged.head()

,state,ev_count,avg_aqi,category
0,Andaman and Nicobar Islands,107,57.706897,Others
1,Andhra Pradesh,99600,77.507701,Others
2,Arunachal Pradesh,72,54.485265,Others
3,Assam,171813,114.117456,Others
4,Bihar,253242,157.158122,Top 5 EV States


In [4]:
files.download("EV_Adoption_vs_AQI.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>